## Huggingface Embedding Model
* 한글을 가장 잘 embedding 할 수 있는 bge-m3 model 을 사용해 문장을 임베딩하고 유사도를 구한다

In [6]:
%pip install -U sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-m3")

bge-m3 embedding 수행

In [3]:
embedded_vector = model.encode("야 저기 차 온다")
embedded_vector.shape

(1024,)

In [5]:
import pandas as pd

data = [
    "내일 차타고 놀러가자",
    "지금 오는 버스는 어디서 오는 버스야?",
    "차 한잔 하면서 이야기 하시죠",
    "5차 공동구매! 오늘만 세일!",
    "홍차 녹차 중에 어떤 차가 좋으세요?",
]

df = pd.DataFrame(data, columns=["text"])
df

,text
0,내일 차타고 놀러가자
1,지금 오는 버스는 어디서 오는 버스야?
2,차 한잔 하면서 이야기 하시죠
3,5차 공동구매! 오늘만 세일!
4,홍차 녹차 중에 어떤 차가 좋으세요?


In [8]:
def get_embedding(text):
    return list(
        model.encode(text)
    )  # bge-m3 encoder 를 이용해 문장에 대한 embedding vector 생성


df["embedding"] = [get_embedding(text) for text in df["text"]]
df

,text,embedding
0,내일 차타고 놀러가자,"[0.0054787057, 0.016985083, -0.020413557, 0.02..."
1,지금 오는 버스는 어디서 오는 버스야?,"[-0.013388535, 0.024189478, -0.02540331, -0.01..."
2,차 한잔 하면서 이야기 하시죠,"[-0.0022158083, 0.010684644, -0.03901669, 0.00..."
3,5차 공동구매! 오늘만 세일!,"[-0.030319396, -0.0011549637, -0.03304445, 0.0..."
4,홍차 녹차 중에 어떤 차가 좋으세요?,"[0.03611525, 0.0046490026, -0.021344088, -0.02..."


## 유사도 구하기
* 벡터의 유사도를 구하는 방법 중 가장 쉽고 간단한 cosine similarity

In [9]:
import numpy as np


def cos_sim(A, B):
    return A @ B / (np.linalg.norm(A) * np.linalg.norm(B))

In [10]:
def return_answer_candidate(df, query):
    """
    df : 문장과 임베딩 벡터가 들어있는 데이터 프레임
    query : 질의할 문장
    """
    query_embedding = get_embedding(query)
    df["similarity"] = df.embedding.apply(
        lambda x: cos_sim(np.array(x), np.array(query_embedding))
    )

    # 입력한 문장과 데이터셋 내에 있는 문장의 유사도를 내림차순 정렬
    results_co = df.sort_values("similarity", ascending=False, ignore_index=True)
    return results_co.head(3)

In [11]:
sim_result = return_answer_candidate(df, "야 저기 차 온다!")
sim_result

,text,embedding,similarity
0,내일 차타고 놀러가자,"[0.0054787057, 0.016985083, -0.020413557, 0.02...",0.707860
1,지금 오는 버스는 어디서 오는 버스야?,"[-0.013388535, 0.024189478, -0.02540331, -0.01...",0.700798
2,차 한잔 하면서 이야기 하시죠,"[-0.0022158083, 0.010684644, -0.03901669, 0.00...",0.632025


In [12]:
sim_result = return_answer_candidate(df, "예쁜 카페 가고 싶어!")
sim_result

,text,embedding,similarity
0,차 한잔 하면서 이야기 하시죠,"[-0.0022158083, 0.010684644, -0.03901669, 0.00...",0.629125
1,내일 차타고 놀러가자,"[0.0054787057, 0.016985083, -0.020413557, 0.02...",0.608427
2,지금 오는 버스는 어디서 오는 버스야?,"[-0.013388535, 0.024189478, -0.02540331, -0.01...",0.530346
